In [1]:
import cx_Oracle as ora
import pandas as pd

In [3]:
dsnStr = ora.makedsn(host="10.60.54.12", port="1521", service_name="DEC_DATA")
con = ora.connect(user="ADM32157HIEUNGUYENPHI", password="ADM32157HIEU", dsn=dsnStr)
c = con.cursor()
start_date = "DATE'2022-12-01'"
end_date = "DATE'2022-12-31'"
query = '''SELECT x.CLIENT_NO
    ,y.IB_REGISTERED_DATE2
    ,TRAN_DATE
    ,TRAN_AMOUNT
FROM 
    (SELECT * FROM DEC_BU.U_PROD_FT_TRAN_LOG
    WHERE TRANS_LV3 = 'Mobile'
    AND STATUS = 'S'
    AND TRAN_DATE BETWEEN :start_date AND :end_date) x
LEFT JOIN DEC_BU.U_TABLEAU_CUSTOMER y
ON x.CLIENT_NO = y.CLIENT_NO'''
query = query.replace(':start_date',start_date).replace(':end_date',end_date)
query 
result = c.execute(query).fetchall()

In [7]:
df = pd.DataFrame(result)
df.columns = [i[0] for i in c.description]
df 

,CLIENT_NO,IB_REGISTERED_DATE2,TRAN_DATE,TRAN_AMOUNT
0,00114749,2022-12-02,2022-12-16,19400
1,00931638,2022-12-09,2022-12-18,48500
2,00931638,2022-12-09,2022-12-18,291000
3,00931638,2022-12-09,2022-12-22,97000
4,00931638,2022-12-09,2022-12-31,97000
...,...,...,...,...
193853,04118003,2022-10-10,2022-12-21,97000
193854,04118003,2022-10-10,2022-12-24,194000
193855,04038526,2022-09-15,2022-12-17,97000
193856,04282337,2022-12-01,2022-12-08,48500


In [10]:
origin = pd.read_csv('MB2_TOPUP_MOBILE.csv', sep = '|')

In [27]:
origin['IB_REGISTERED_DATE2'] = pd.to_datetime(origin['IB_REGISTERED_DATE2'])
origin['TRAN_DATE'] = pd.to_datetime(origin['TRAN_DATE'])
origin['IB_REGISTERED_DATE2'] = origin['IB_REGISTERED_DATE2'].dt.date
origin['TRAN_DATE'] = origin['TRAN_DATE'].dt.date
origin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 337844 entries, 0 to 337843
Data columns (total 4 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   CLIENT_NO            337844 non-null  int64 
 1   IB_REGISTERED_DATE2  337844 non-null  object
 2   TRAN_DATE            337844 non-null  object
 3   TRAN_AMOUNT          337844 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 10.3+ MB


In [33]:
df['CLIENT_NO'] = df['CLIENT_NO'].astype(int)
df['IB_REGISTERED_DATE2'] = df['IB_REGISTERED_DATE2'].dt.date
df['TRAN_DATE'] = df['TRAN_DATE'].dt.date
df.info()

AttributeError: Can only use .dt accessor with datetimelike values

In [34]:
origin_ = origin.append(df)
origin_.shape

(531702, 4)

In [35]:
origin_ = origin_.reset_index(drop=True)
origin_

,CLIENT_NO,IB_REGISTERED_DATE2,TRAN_DATE,TRAN_AMOUNT
0,190011,2012-11-21,2022-07-14,300000
1,190011,2012-11-21,2022-07-11,100000
2,190011,2012-11-21,2022-08-08,100000
3,190011,2012-11-21,2022-08-05,100000
4,190011,2012-11-21,2022-10-28,485000
...,...,...,...,...
531697,4118003,2022-10-10,2022-12-21,97000
531698,4118003,2022-10-10,2022-12-24,194000
531699,4038526,2022-09-15,2022-12-17,97000
531700,4282337,2022-12-01,2022-12-08,48500


In [36]:
origin_.to_csv('MB2_TOPUP_MOBILE.csv', index=False)